# Object Detection Model

Transfer Learning Model for fine-tuning YOLO on Coca-Cola cans

---

## Inizialitation 📚

In [15]:
# Libraries
import os
from ultralytics import YOLO
import yaml
import torch
from roboflow import Roboflow

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Set main directory
main_dir = "C:/Users/julia/Trabajo/IFAI Solutions/Repositorios/YOLO_Object-Detection"
os.chdir(main_dir)

In [11]:
# Model folder
folder_path = "model"

# Create folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created")

else:
    print(f"Folder '{folder_path}' already exists")

Folder 'model' already exists


In [ ]:
# GPU verification
print(f"Setup completed. Using torch {torch.__version__}")

if torch.cuda.is_available():
    print(f"\n ✅ Nice! GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("\n ⚠️ No GPU available. Training will run on CPU, which will be slower.")

Setup completed. Using torch 2.9.1+cpu

 ⚠️ No GPU available. Training will run on CPU, which will be slower.


## Loading the dataset ℹ️

In [ ]:
# Download pre-trained model
model = YOLO(os.path.join(folder_path, "yolov8n.pt")) # Nano version "m" (also you can use "s" and "m")

print("Model loaded successfully:\n")

model.info()

Model loaded successfully:

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [19]:
# Roboflow client (API key from .env)
rf = Roboflow(api_key=os.getenv("roboflow_api_key"))

# Project
project = rf.workspace("ifai-solutions").project("coca-cola-deteccion-de-logo-syvmz")

# Download dataset
version = project.version(3)

os.chdir(folder_path)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Coca-Cola---Detección-de-logo-3 in yolov8:: 100%|██████████| 572/572 [00:00<00:00, 1427.54it/s]


## Model training 💪🏻

In [24]:
# Load a model
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data='Coca-Cola---Detección-de-logo-3/data.yaml',   # yaml with dataset info
    epochs=30, # Number of epochs
    imgsz=640, # Image size (YOLO default)
    batch=16, # Batch size
    patience=10, # Stop if not improving
    name='CocaCola_Model', # Name of your results folder
    exist_ok=True, # Don't throw error if folder already exists
    device=None # Use GPU if available. If not, use CPU
)

Ultralytics 8.3.231  Python-3.13.5 torch-2.9.1+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Coca-Cola---Deteccin-de-logo-3/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=CocaCola_Model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, pers

## Predictions 🔮

In [26]:
# Change dir
os.chdir(main_dir)

In [28]:
# Best model
best_model = YOLO("runs/detect/CocaCola_Model/weights/best.pt")

# Test images folder
test_folder = "model/Coca-Cola---Detección-de-logo-3/test/images"

# Predictions over test images
results = best_model.predict(test_folder)


image 1/10 C:\Users\julia\Trabajo\IFAI Solutions\Repositorios\YOLO_Object-Detection\model\Coca-Cola---Deteccin-de-logo-3\test\images\244678885_288639169764068_3569159260879958382_n_jpg.rf.15a31724185cd9666f402eb331e8b142.jpg: 640x640 1 object, 77.5ms
image 2/10 C:\Users\julia\Trabajo\IFAI Solutions\Repositorios\YOLO_Object-Detection\model\Coca-Cola---Deteccin-de-logo-3\test\images\260059958_416351856823506_8159905141872482313_n_jpg.rf.6632514581945b36deebe38ac4bb0697.jpg: 640x640 3 objects, 62.0ms
image 3/10 C:\Users\julia\Trabajo\IFAI Solutions\Repositorios\YOLO_Object-Detection\model\Coca-Cola---Deteccin-de-logo-3\test\images\260099231_1010998619757147_2273483457060344948_n_jpg.rf.b9b0a445eb4ebee322d1739d317cca90.jpg: 640x640 3 objects, 68.6ms
image 4/10 C:\Users\julia\Trabajo\IFAI Solutions\Repositorios\YOLO_Object-Detection\model\Coca-Cola---Deteccin-de-logo-3\test\images\260112505_444253450590660_1636461285321175621_n_jpg.rf.aab76e7f7fb5db35ee6d8461f378c604.jpg: 640x640 2 objects

In [ ]:
# Visualize predictions
for img in results:
    img.show()